# S04: Extracting Patch Features

Here we utilize a image feature extractor to extract deep features from all the patches that we have obtained from the step `S03`. The image feature extractor used in this step is a ResNet50 pretrained on ImageNet. 

We also adopt `CLAM` (https://github.com/mahmoodlab/CLAM) for the above purpose. 

## 1. My notes

In this step, for each slide, its patch coordinates, which are stored in a `h5` file (see `ROOT_DIR_FOR_DATA_SAVING/tiles-l1-s256/patches/` in your server), would be loaded and then used to locate certain patch regions (each with the size you specified in the step `S03`) in this slide image (at the magnification you specified in the step `S03`). Meanwhile, the source file of the slide will also be loaded for reading patch regions. At the end, all patch features of the slide will be saved in a `pt` file. 

Key codes are as follows:

```python
# key codes for calculating patch features
all_feats = None
all_coors = None
for count, (batch, coords) in enumerate(loader):
    coords = torch.from_numpy(coords)
    with torch.no_grad():   
        if count % print_every == 0:
            print('batch {}/{}, {} files processed'.format(count, len(loader), count * batch_size))
        batch = batch.to(device, non_blocking=True)
        mini_bs = coords.shape[0]

        features = model(batch).cpu()
        if all_feats is None:
            all_feats = features
            all_coors = coords
        else:
            all_feats = torch.cat([all_feats, features], axis=0)
            all_coors = torch.cat([all_coors, coords], axis=0)
```

## 2. Running feature extraction

Similarly, there are also two options for you. The first one, official CLAM, is recommended. The second one, our improved CLAM, would requires extra learning costs.

### 2.1 Using official CLAM

Please refer to https://github.com/mahmoodlab/CLAM for the details of patch feature extraction.

### 2.2 Using an improved version of CLAM

For this step, we have improved CLAM specifically in 
- different architectures for feature extracting,
- and patch color normalization

A detailed bash script (placed at `./tools/scripts/S04-Extracting-Feats.sh`) is as follows:

```bash
#!/bin/bash
set -e

# Sample patches of SIZE x SIZE at LEVEL 
LEVEL=1
SIZE=256

# Path where CLAM is installed
DIR_REPO=../CLAM

# Root path to pathology images 
DIR_RAW_DATA=/NAS02/RawData/tcga_rcc
DIR_EXP_DATA=/NAS02/ExpData/tcga_rcc

# Sub-directory to the patch coordinates generated from S03
SUBDIR_READ=tiles-l${LEVEL}-s${SIZE}

# Sub-directory to the patch features 
SUBDIR_SAVE=feats-l${LEVEL}-s${SIZE}-RN50-color_norm

cd ${DIR_REPO}

echo "running for extracting features from all tiles"
CUDA_VISIBLE_DEVICES=1 python3 extract_features_fp.py \
    --data_h5_dir ${DIR_EXP_DATA}/${SUBDIR_READ} \
    --data_slide_dir ${DIR_RAW_DATA} \
    --csv_path ${DIR_EXP_DATA}/${SUBDIR_READ}/process_list_autogen.csv \
    --feat_dir ${DIR_EXP_DATA}/${SUBDIR_SAVE} \
    --batch_size 128 \
    --slide_ext .svs \
    --color_norm \
    --slide_in_child_dir --no_auto_skip
```

You could run this script using the following command:
```bash
nohup ./S04-Extracting-Feats.sh > S04-Extract-Feats.log 2>&1 &
```

Full running logs could be found in `./tools/scripts/S04-Extract-Feats.log`. 

Next, we check if the number of generated files is consistent with that of patch files from the step `S03`.

In [1]:
import os
import os.path as osp

DIR_FEAT = "/NAS02/ExpData/tcga_rcc/feats-l1-s256-RN50-color_norm/pt_files"
feat_files = [f for f in os.listdir(DIR_FEAT) if f.endswith(".pt")]
print("This step generated {} feature files in {}.".format(len(feat_files), DIR_FEAT))

This step generated 937 feature files in /NAS02/ExpData/tcga_rcc/feats-l1-s256-RN50-color_norm/pt_files.


In [2]:
DIR_PATCH = "/NAS02/ExpData/tcga_rcc/tiles-l1-s256/patches"
patch_files = [f for f in os.listdir(DIR_PATCH) if f.endswith(".h5")]
print("The step S03 generated {} patch files in {}.".format(len(patch_files), DIR_PATCH))

The step S03 generated 937 patch files in /NAS02/ExpData/tcga_rcc/tiles-l1-s256/patches.


In [3]:
feat_filenames = [osp.splitext(f)[0] for f in feat_files]
patch_filenames = [osp.splitext(f)[0] for f in patch_files]
flag = False
for f in patch_filenames:
    if f not in feat_filenames:
        flag = True
        print("Expected {}, but it was not found in features files.".format(f))
if flag:
    print("Some slides were not processed.")
else:
    print("All slides in patch directory have been processed in this step.")

All slides in patch directory have been processed in this step.



### 2.3 An example

The running log of the first WSI is presented as follows:

```txt
progress: 0/940
TCGA-2K-A9WE-01Z-00-DX1.ED8ADE3B-D49B-403B-B4EB-BD11D91DD676
downsample [4.00005125 4.00008641]
downsampled_level_dim [39021 23146]
level_dim [39021 23146]
name TCGA-2K-A9WE-01Z-00-DX1.ED8ADE3B-D49B-403B-B4EB-BD11D91DD676
patch_level 1
patch_size 256
save_path /NAS02/ExpData/tcga_rcc/tiles-l1-s256/patches

feature extraction settings:
-- target patch size:  None
-- pretrained:  True
-- patches sampler: None
-- color normalization: <torchstain.normalizers.torch_macenko_normalizer.TorchMacenkoNormalizer object at 0x7f92bde73f98>
-- color argmentation: None
-- add_patch_noise: None
-- transformations:  Compose(
    Lambda()
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
)
processing /NAS02/ExpData/tcga_rcc/tiles-l1-s256/patches/TCGA-2K-A9WE-01Z-00-DX1.ED8ADE3B-D49B-403B-B4EB-BD11D91DD676.h5: total of 57 batches
batch 0/57, 0 files processed
batch 20/57, 2560 files processed
batch 40/57, 5120 files processed
features size:  torch.Size([7274, 1024])
saved pt file:  /NAS02/ExpData/tcga_rcc/feats-l1-s256-RN50-color_norm/pt_files/TCGA-2K-A9WE-01Z-00-DX1.ED8ADE3B-D49B-403B-B4EB-BD11D91DD676.pt

computing features for /NAS02/ExpData/tcga_rcc/feats-l1-s256-RN50-color_norm/pt_files/TCGA-2K-A9WE-01Z-00-DX1.ED8ADE3B-D49B-403B-B4EB-BD11D91DD676.pt took 75.39572024345398 s
```